Before I go onto finding the vegetation difference between two images from the sentinal 1 satellite. One from 2008 and another from 2020. In order to see the benefit of the great green wall. I need to read all about what SAR satellites are here: https://elib.dlr.de/82313/1/SAR-Tutorial-March-2013.pdf. And complete the part 3 of the GEE tutorial. It would also be prudent to go over the statistics in part 1 and 2. The statistics seem fairly simple but it is there implications that I am struggling a bit with.

In [23]:
# My jupyter path on my mac is different to pip, the easy answer is to append the path to where my packages are inported
import sys
sys.path.append("/Users/calummeikle/Documents/ClimateModelling/SatelliteImagery/env/lib/python3.9/site-packages/")
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages')

In [24]:
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp

In [25]:
import ee
 
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AY0e-g7bLlwI-buxdog_DxsSrFhnpTjF9awfhuB_-8uRhEO7BaNRj6L3T98

Successfully saved authorization token.


In [33]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [47]:
# A geojson for the Tokyo area taken from geojson.io
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              139.2572021484375,
              35.420391545750746
            ],
            [
              140.18554687499997,
              35.420391545750746
            ],
            [
              140.18554687499997,
              35.98245135784044
            ],
            [
              139.2572021484375,
              35.98245135784044
            ],
            [
              139.2572021484375,
              35.420391545750746
            ]
          ]
        ]
      }
    }
  ]
}

In [48]:
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [49]:
im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-06-01'), ee.Date('2020-08-31')))

In [50]:
acq_times = im_coll.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

['06/02/20',
 '06/14/20',
 '06/26/20',
 '07/08/20',
 '07/20/20',
 '08/01/20',
 '08/13/20',
 '08/25/20',
 '06/03/20',
 '06/08/20',
 '06/08/20',
 '06/15/20',
 '06/20/20',
 '06/20/20',
 '06/27/20',
 '07/02/20',
 '07/02/20',
 '07/09/20',
 '07/14/20',
 '07/14/20',
 '07/21/20',
 '07/26/20',
 '07/26/20',
 '08/02/20',
 '08/07/20',
 '08/07/20',
 '08/14/20',
 '08/19/20',
 '08/19/20',
 '08/26/20']

In [51]:
# First images on GEE 2014 which corrasponds quite closely with the 

In [52]:
def clip_img(img):
    """Clips a list of images."""
    return ee.Image(img).clip(aoi)

im_list = im_coll.toList(im_coll.size())
im_list = ee.List(im_list.map(clip_img))

im_list.length().getInfo()

30

In [53]:
def selectvv(current):
    return ee.Image(current).select('VV')

vv_list = im_list.map(selectvv)

In [55]:
location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(location=location, zoom_start=11)
rgb_images = (ee.Image.rgb(vv_list.get(0), vv_list.get(1), vv_list.get(2)).log10().multiply(10))
mp.add_ee_layer(vv_list.get(0), {'min': -20,'max': 0}, 'VV 0')
mp.add_ee_layer(vv_list.get(1), {'min': -20,'max': 0}, 'VV 1')
mp.add_ee_layer(vv_list.get(2), {'min': -20,'max': 0}, 'VV 2')
mp.add_ee_layer(rgb_images, {'min': -20,'max': 0}, 'rgb composite') 
mp.add_child(folium.LayerControl())

Above is 4 images overlaid on a Folium map. VV 0, 1 and 2 were taken 12 days apart and the rgb composite is those three images superimposed to see any changes. As you can see there is none really, as you'd expect. However I wonder if we could see a change if we did the same analysis after a big typhoon, like typhoon Hagibis which hit in early October 2019.

In [63]:
im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2019-09-01'), ee.Date('2019-11-01')))

In [64]:
acq_times = im_coll.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

['09/12/19',
 '09/24/19',
 '10/06/19',
 '10/13/19',
 '10/18/19',
 '10/18/19',
 '10/18/19',
 '10/30/19',
 '09/01/19',
 '09/06/19',
 '09/06/19',
 '09/13/19',
 '09/18/19',
 '09/18/19',
 '09/25/19',
 '09/30/19',
 '09/30/19',
 '10/07/19',
 '10/12/19',
 '10/12/19',
 '10/19/19',
 '10/24/19',
 '10/24/19',
 '10/31/19']

In [72]:
im_list = im_coll.toList(im_coll.size())
im_list = ee.List(im_list.map(clip_img))
vv_list = im_list.map(selectvv)
im_list.length().getInfo()

24

In [82]:
location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(location=location, zoom_start=11)
before_typhoon = ee.Image(vv_list.get(2)).clip(aoi)
after_typhoon = ee.Image(vv_list.get(5)).clip(aoi)
ratio = before_typhoon.divide(after_typhoon)
mp.add_ee_layer(vv_list.get(2), {'min': -20,'max': 0}, 'VV 2')
mp.add_ee_layer(vv_list.get(5), {'min': -20,'max': 0}, 'VV 5')
mp.add_ee_layer(ratio, {'min': 0,'max': 20}, 'Ratio')
mp.add_child(folium.LayerControl())

There is a definite difference between the images. But the ratio seems too uncorrelated to have produced any significant change. And I think the search area is too large. Possibly next time find an area or building that was known to be damaged by the typhoon, and see if we can find any change in those SAR images.

In [83]:
# TODO: Calculate the statistics